In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython import display as ds
import matplotlib.image as mpimg
import cv2
import tensorflow as tf

%matplotlib inline

/home/kvpcloud/.conda/envs/p3_gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Dataset:
    main = 'dataset/UCSD_Anomaly_Dataset.v1p2'
    latent_train = np.load('{}/latent_train.npy'.format(main))
    #test_images = np.load('{}/test.npy'.format(main))

In [3]:
print(Dataset.latent_train)

[[0.09196392 0.0478623  0.13000081 ... 0.10197813 0.         0.03637807]
 [0.11980576 0.07599361 0.08409608 ... 0.11578386 0.         0.01681237]
 [0.04043263 0.06748637 0.09166412 ... 0.10462585 0.         0.04567996]
 ...
 [0.07251541 0.06275192 0.06756014 ... 0.07906516 0.         0.05174178]
 [0.05823517 0.05963557 0.09027178 ... 0.10724749 0.         0.0523259 ]
 [0.12807868 0.10718347 0.12245025 ... 0.07215846 0.         0.03251233]]


In [4]:
print(Dataset.latent_train.shape)

(200, 300)


In [5]:
class Params:
    latent_feature_count = [10,10,3]
    epochs = 1
    input_batch_size = 200
    output_size = 1

In [10]:
class Network:
    
    def __init__(self):
        self.latent = tf.placeholder(tf.float32, shape=[Params.input_batch_size] + [np.prod(Params.latent_feature_count)], name='x')
        print('Input Latent', self.latent.get_shape())
        hidden_state = tf.zeros([Params.output_size, np.prod(Params.latent_feature_count)])
        current_state = tf.zeros([Params.output_size, np.prod(Params.latent_feature_count)])
        self.state = hidden_state, current_state
        
        
    def build_network(self):
        
        y_list = []
        
        with tf.name_scope('LSTM') as lstm_scope:
            x = tf.layers.dense(self.latent, units=np.prod(Params.latent_feature_count), activation=tf.nn.relu)
            
            lstm = tf.contrib.rnn.BasicLSTMCell(512) #np.prod(Params.latent_feature_count))
            x_list = tf.split(x, 10, axis=0)
            #print("Mini Batch ", x_list.get_shape())
            

            y_list, self.state = tf.nn.static_rnn(lstm, x_list, dtype=tf.float32)
            
            self.y = y_list[-1]
            self.y = tf.layers.dense(self.y, np.prod(Params.latent_feature_count))
                
            print('Output Latent', self.y.get_shape())
        
    def get_latent(self):
        return self.y

In [11]:
with tf.Session() as sess:
    network=Network()
    network.build_network()
    y = network.get_latent()
    sess.run(tf.global_variables_initializer())
    
    Dataset.y = sess.run(y, feed_dict={network.latent:Dataset.latent_train})

Input Latent (200, 300)
Output Latent (20, 300)


In [ ]:
Dataset.y

In [ ]:
Dataset.y.shape